# INTRODUCTION TO JULIA

## Warning

+ Setting: Phd seminar in Prague 2011. MK talks about _FEniCS_. Wrongly, people fall asleep. __FEniCS is awesome__.
+ Setting: Phd seminar in Oslo 2016. MK talks about _Julia_. ???. __Julia is awesome__.

## Setup

+ Ubuntu package from the repository
```
sudo add-apt-repository ppa:staticfloat/juliareleases
sudo add-apt-repository ppa:staticfloat/julia-deps
sudo apt-get update
sudo apt-get install julia
```
+ [Download](http://julialang.org/downloads/) binaries (all platforms)
+ Build from source code
```
git clone https://github.com/JuliaLang/julia.git
cd julia
git checkout release-0.5  # unless you want the latest master
make -j 4
```
+ Try Julia in your browser via [JuliaBox](https://www.juliabox.org/). No installation but Google account required

## First steps inside the notebook

In [ ]:
# Note that (unlike numpy) the vectors are column vectors, not row vectors
A = [1, 2, 3, 4]

In [ ]:
# 1-based indexing as if matlab
first(A) == A[1]

In [ ]:
# Some other matlab influence
A.^2   # not A**2

In [ ]:
# Just showing off unicode
2 ∈ A

In [ ]:
# Checking subsets
b = Set([2, 3])
a = Set(A)

Same as in Python we can can if b is a subset of a as `b < a`. But if you got used on the beuty of $\in$ you might find it hard to settle for `<`. So let's define our first function 

In [ ]:
⊂(a, b) = a < b

In [ ]:
# Testdrive
b < a && b ⊂ a 

## What is Julia ?
+ Julia: A fresh approach to numerical computing, see [paper](http://arxiv.org/abs/1411.1607)
+ High level, dynamic, (dynamically) typed, general purpose language
+ Free and open-source. MIT licence
+ Went [viral](https://www.google.com/trends/explore#q=julia%20language) on Valentine's day 2014
+ Some features we will see in action today
    - type inference
    - JIT compiler
    - rich typestem
    - multiple dispatch
    - zero overhead C/Fortran calls 
    - metaprogramming
+ An important feature missing today is _built in parallelism_ (distributed memory)

## Why do we need yet another language?

Short answer - to address the two language paradigm
+ Express the idea in high level language(Python) - setup prototype quickly. Performance far from production code
+ Production code then written in low level language (C/C++)
+ Write performance critical parts of code in low level language and glue together ([Numba](http://numba.pydata.org/), SWIG)
    - FEniCS uses Python to generate C++ code (see poisson.ufl, poisson.h) that is compiled and wrapped for Python
    - Diako/[Mikael](http://www.sciencedirect.com/science/article/pii/S0010465516300200) prototype in Python/NumPy and then write the same code in [Cython](https://github.com/spectralDNS/spectralDNS/blob/master/spectralDNS/optimization/cython_maths.in)
+ To extend NumPy/SciPy you (usually) have to write C

Julia's approach
+ Designed to be easy to write with performance close to C
+ If that is true you need one language to solve your problem
+ It seems to be true: benchmarks from the Julia website

# Some benchmark problems of our own

## Project Euler problem 14. 
The following iterative sequence is defined for the set of positive integers:

+ n → n/2 (n is even)
+ n → 3n + 1 (n is odd)

Using the rule above and starting with 13, we generate the following sequence: 13 → 40 → 20 → 10 → 5 → 16 → 8 → 4 → 2 → 1. Which starting number, under one million, produces the longest chain? Note that whether the chain terminates for arbitrary $n$ is an unproved conjecture.

+ To solve PE problem's the solution must be obtained in less then 60s. Honor code is to beat 1s.
+ Note that the code presented here is a bruteforce solution
+ The answer it produces is correct

In [ ]:
"""Compute the Collatz chain for number n."""
function collatz_chain(n)
    k = 1
    while n > 1
        n = isodd(n) ? 3n+1 : n >> 1
        k += 1
        # println(n)
    end
    k
end

"""Which of the number [1, stop) has the longest Collatz chain."""
function solve_euler(stop)
    n, N, N_max = 1, 0, 0  
    while n < stop
        value = collatz_chain(n)
        if value > N_max
            N = n
            N_max = value
        end
        n += 1
    end
    (N, N_max)
end

In [ ]:
# Let's time it
N = 1000000
t0 = tic()
answer = solve_euler(N)
t1 = toc();
answer, t1

In [ ]:
@time solve_euler(N)

Julia is done in about 0.5s, Python need ~24s. So with Julia I am onto the next problem. With Python it is back to the drawing board.

## Julia fractal
+ It is only approprate that we explore the Julia set
+ Explore points $z$ in the complex plain such that $z_{n+1}=z^2_{n}+c$ is bounded for any n

In [ ]:
# Julia code is a translation of the definition
"""Color Julia set. Code from https://www.youtube.com/watch?v=PsjANO10KgM"""
function julia(z, c)
    for n in 1:80
        if abs2(z) > 4
            return n-1
        end
        z = z*z + c
    end
    return 80
end

"""Color Julia set. Prealoc version"""
function julia(x, y, c)
    J = zeros(length(x), length(y))
    index = 0
    for r in y, i in x
        index += 1
        J[index] = julia(complex(r, i), c)
    end
    J
end

In [ ]:
# We explore a few points
cs = (complex(-0.06, 0.67), complex(0.279, 0), complex(-0.4, 0.6), complex(0.285, 0.01))

x = collect(1:-0.002:-1);
y = collect(-1.5:0.002:1.5);

Js = []
# Evaluate fractal generation
t0 = tic()
for c in cs
    # No prealoc 
    push!(Js, [julia(complex(r, i), c) for i in x, r in y]);
    # Prealoc out
    push!(Js, julia(x, y, c));
end
t1 = toc()

@printf("Generated in %.4f s\n", t1);
println("Image size $(size(Js[1], 1))x$(size(Js[1], 2))")

In [ ]:
# Here we use python interopt(more on that later) to plot the fractals
using PyPlot

for J in Js
    figure(figsize=(12, 8))
    imshow(J, cmap="viridis", extent=[-1.5, 1.5, -1, 1])
end
show()

Python requires close to 5s to generate the fractals. With either of the Julia versions the execution time is close to 1s. Note that at this point we have two `julia` methods. Which one gets called when the code is run is our first encounter with _multiple dispatch_.